In [50]:
import pandas as pd
import numpy as np
import mitosheet
import json

In [11]:
input_data = "../5a-Matching/output/matched_processed.csv"
df_input = pd.read_csv(input_data, low_memory=False, dtype="str")

In [95]:
provinces = df_input['province'].unique()

IDX = []
ST_NO = []
ST_NAME = []
ST_NAME_2 = []
CITY = []
PROV = []
POST = []
LATS = []
LONS = []

df = pd.DataFrame(columns=['idx','gc_street_no','gc_street_name', \
                               'gc_city','gc_province','gc_postal_code', \
                               'gc_latitude', 'gc_longitude'])

for province_code in provinces:
    print(province_code)
    json_file = 'data/geocoded_' + str(province_code) + '.json'

    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            JSONS=json.load(f)  
        print(len(JSONS))

        # add json data to arrays
        for index, element in enumerate(JSONS):
            if element==[]:
                append_blank(index)

            else:
                # format gc api results      
                if 'meta' in element:
                    if (element['data'][0]['country']['code'] == 'CA'):
                        IDX.append(element['idx'])
                        if not element['data'][0]['civicNumber']['number'] is None:
                            ST_NO.append(element['data'][0]['civicNumber']['number'])
                        else:
                            ST_NO.append('')
                            print('oops')
                        ST_NAME.append(element['data'][0]['streetName'])
                        ST_NAME_2.append(element['data'][1]['streetName'])
                        CITY.append(element['data'][0]['cityName']['en'])
                        PROV.append(element['data'][0]['mailingAddress']['province']['code'])
                        POST.append(element['data'][0]['mailingAddress']['postalCode'])
                        LATS.append(element['data'][0]['location']['geoCoordinates']['latitude'])
                        LONS.append(element['data'][0]['location']['geoCoordinates']['longitude'])
        #                 if 'streetName' in element.keys():
        #                     ST_NAME.append(element['streetName'])
        #                 else:
        #                     ST_NAME.append('nope')
                else:
                    IDX.append('')
                    LATS.append('')
                    LONS.append('')
                    ST_NO.append('nope')
                    ST_NAME.append('')
                    CITY.append('')
                    PROV.append('')
                    POST.append('')
                    ST_NAME_2.append('')
    except FileNotFoundError:
        print('no file: ', json_file)
        
    
    

# append results to dataframe
df['idx'] = IDX
df['gc_street_no'] = ST_NO
df['gc_street_name'] = ST_NAME
df['gc_street_name_2'] = ST_NAME_2
df['gc_city'] = CITY
df['gc_province'] = PROV
df['gc_postal_code'] = POST
df['gc_latitude'] = LATS
df['gc_longitude'] = LONS

78
no file:  data/geocoded_78.json
AB
216
BC
14042
LAKEVIEW ESTATES
no file:  data/geocoded_LAKEVIEW ESTATES.json
MB
47
NB
19
NL
2
NS
5
NT
81
NU
no file:  data/geocoded_NU.json
ON
19893
PE
1
QC
122758
SK
66
YT
3
nan


TypeError: can only concatenate str (not "float") to str

In [83]:
df2 = pd.merge(df_input, df, how='left' on='idx')
len(df2)

14037

In [84]:
df2 = df2[['idx', 'street_no', 'gc_street_no', 'street_name', 
          'gc_street_name', 'gc_street_name_2', 'latitude', 'gc_latitude', 
          'longitude', 'gc_longitude']]
mitosheet.sheet(df2, analysis_to_replay="id-jtutpdnplx")


MitoWidget(analysis_data_json='{"analysisName": "id-uosvattzwc", "analysisToReplay": {"analysisName": "id-jtut…

In [85]:
# how many look good?

# if the street numbers match
# and the gc_street_name is contained in street_name
for i in range(len(df2)):
    if (df2.at[i,'street_no'] == df2.at[i,'gc_street_no'] and
       df2.at[i,'gc_street_name'].lower() in df2.at[i,'street_name'].lower()):
        df2.at[i,'keep_gc'] = 'yes'


In [86]:
total = len(df2)
keeps = len(df2[df2['keep_gc'] == 'yes'])
percent_matches = 100 * keeps / total
print('percent of good results: ', np.round(percent_matches, 0))

percent of good results:  61.0
